<a href="https://colab.research.google.com/github/camstarhall/KF5042/blob/main/Hate_Speech_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import nltk
import pandas as pd
import numpy as np

from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [4]:
# set the data path and import the dataset using pandas

dataPath = '/content/labeled_data.csv'
df = pd.read_csv(dataPath)

# only the class and tweet data is needed

df = df[['class', 'tweet']]
df.head()

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [5]:
# text preprocessing
# use the nltk built in tweet tokenizer

tknzr = TweetTokenizer()
df['tweet'] = df['tweet'].apply(lambda x: tknzr.tokenize(x))

# lowercasing of tweet text

df['tweet'] = df['tweet'].apply(lambda x: [word.lower() for word in x])

# removal of stop words (also from nltk)

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
df['tweet'] = df['tweet'].apply(lambda x: [word for word in x if word not in stop_words])

# remove non alphabetic characters

import string
punctuations = list(string.punctuation)
df['tweet'] = df['tweet'].apply(lambda x: [word for word in x if word.isalpha()])

# lemmatization

nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
df['tweet'] = df['tweet'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

# as the data is from tweets, the text contains the word 'rt' unneccesarily
# this line just removes this

df['tweet'] = df['tweet'].apply(lambda x: [word for word in x if word != 'rt'])

# finish processing an return to string

df['tweet'] = df['tweet'].apply(lambda x: ' '.join(x))

df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


,class,tweet
0,2,woman complain cleaning house man always take ...
1,1,boy dat cold tyga dwn bad cuffin dat hoe place
2,1,dawg ever fuck bitch start cry confused shit
3,1,look like tranny
4,1,shit hear might true might faker bitch told ya


To do from here:

- Feature extraction
- Training / testing split
- Training
- Parameter Tuning

In [6]:
from sklearn.model_selection import train_test_split

# split data into 80 / 10 / 10 for training / testing / valitating
# first split off the 20% (test_size=0.2)
train_texts, test_texts, train_labels, test_labels = train_test_split(df['tweet'], df['class'], test_size=0.2, random_state=42)

# make the split for validation
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.125, random_state=42)  # 0.125 x 0.8 = 0.1

In [7]:
# setting up bag-of-words

from sklearn.feature_extraction.text import CountVectorizer

# initialize vectorizer, set max features to 1000 - adjust this later if needed
vectorizer = CountVectorizer(max_features=1000)

# 'fit' (apply) vectorizer on to training data
X_train = vectorizer.fit_transform(train_texts).toarray()

# do the same for testing / validation data
X_val = vectorizer.transform(val_texts).toarray()
X_test = vectorizer.transform(test_texts).toarray()

# labels on this dataset are already numberical so no encoding needed


In [8]:
### TRAINING TIME

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# set up. a regression model
model = LogisticRegression(max_iter=1000)

# train model
model.fit(X_train, train_labels)

# apply predictions to testing set
val_pred = model.predict(X_val)

# print eval
print("Validation Accuracy:", accuracy_score(val_labels, val_pred))

Validation Accuracy: 0.9039935457845906
